In [12]:
from IPython.display import display
import pandas as pd
import re
from bs4 import BeautifulSoup as b_s
import pandas as pd
import urllib.request as req
import requests
import os
from datetime import datetime as dt
import time
from googleapiclient.discovery import build
import random


today = dt.today().strftime('%Y-%m-%d-%A')
today

def today_weather():
    html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
    soup = b_s(html.text,'html.parser')
    Wdata = soup.find('div', {'class':'weather_box'})
    find_lc = Wdata.find('span',{'class':'btn_select'}).text
    print('현재위치 : '+find_lc,'℃' )
    find_ct = Wdata.find('span',{'class':'todaytemp'}).text
    print("현재온도 : "+find_ct,'℃')
    find_ht = Wdata.select_one('span.max>span.num').text
    print("최고온도 : "+find_ht,'℃')
    find_lt = Wdata.select_one('span.min>span.num').text
    print('최저온도 : '+find_lt,'℃')
    find_ft = Wdata.select_one('span.sensible>em>span.num').text
    print('체감온도 : '+find_ft,'℃')
    find_dt = Wdata.select_one('div.today_area._mainTabContent > div.main_info > div > ul > li:nth-child(1) > p').text
    print(find_dt)
    find_am_hm = Wdata.select_one('li>span.point_time.morning>span.rain_rate>span.num').text
    print('오전강수확률 : '+find_am_hm,'%')
    find_pm_hm = Wdata.select_one('li>span.point_time.afternoon>span.rain_rate>span.num').text
    print('오후강수확률 : '+find_pm_hm,'%')
    
    
def dust():
    html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
    soup = b_s(html.text,'html.parser')
    Ddata = soup.find('div', {'class':'weather_box'})
    find_du = Ddata.select_one('dl.indicator>dd>span.num').text
    ev_du = int(find_du.strip('㎍/㎥'))
    if ev_du <= 30:
        ev_dust = "좋음"
    elif 30 <ev_du<= 80:
        ev_dust = "보통"
    elif 80 <ev_du<= 150:
        ev_dust = "나쁨"
    else:
        ev_dust = "매우나쁨"

    find_M_du = Ddata.select('dl.indicator>dd>span.num')[1].text
    ev_M_du = find_M_du.strip('㎍/㎥')

    try: 
        ev_M_du = int(ev_M_du)
        if ev_M_du <= 15:
            ev_M_dust = "좋음"
        elif 15 <ev_M_du<= 50:
            ev_M_dust = "보통"
        elif 51 <ev_M_du<= 100:
            ev_M_dust = "나쁨"
        else:
            ev_M_dust = "매우나쁨"
    except ValueError as exception:
        ev_M_dust = "너무좋음"
    
    
def corona19():
    html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%BD%94%EB%A1%9C%EB%82%98')
    soup = b_s(html.text,'html.parser')
    html1 = requests.get('http://ncov.mohw.go.kr/')
    soup1 = b_s(html1.text,'html.parser')
    status = soup1.select_one('div.liveNumOuter>h2>a').text
    status = status.split('자세히')[0]
    # type(status)
    status = status.split(',')
    status
    print(status)
    tsct = soup1.select_one('div.datalist>ul>li>span.subtit').string
    tsctn = soup1.select_one('div.datalist>ul>li>span.data').string
    print(tsct,':', tsctn)
    find_cf = soup.select_one('ul>li.info_01>p.info_num').text
    print('누적환자 : ',find_cf,'명')
    
def popular_stock():
    req = requests.get('https://finance.naver.com/sise/')
    html = req.text
    soup = b_s(html,'html.parser')

    stock_names = soup.select('ul.lst_pop>li>a')
#     for stock_name in stock_names:
#         print(stock_name.text)

    stock_costs = soup.select('ul.lst_pop>li>span')
#     for stock_cost in stock_costs:
#         print(stock_cost.text)
    stock_costs
    
    stock_vars = soup.select('ul.lst_pop>li>em.bu_p>span.blind')
#     for stocks_var in stocks_vars:
#         print(stocks_var.string)

    stock = pd.DataFrame([stock_names,stock_costs,stock_vars])
    stock = stock.transpose()
    stock = stock.rename(columns={0:'종목명',1:'현재가',2:'변화'})
    return display(stock)


def NEWS():
    url = 'https://news.naver.com/'
    headers = {'User-Agent':'Mozilla/5.0 (Windosws NT 6.3; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    res = requests.get(url,headers=headers)
    news_soup = b_s(res.text,'html.parser')
    headline_titles = news_soup.select('div.hdline_news>ul.hdline_article_list>li>div.hdline_article_tit')
    headline_title = []
    for title in headline_titles:
        headline_title.append(title.text.strip())
    headline_title

    headline_links = news_soup.select('div.hdline_news>ul.hdline_article_list>li>div.hdline_article_tit>a')
    headline_link = []
    for link in headline_links:
        headline_link.append("https://news.naver.com"+link.get('href'))
    headline_link

    headline_news = pd.DataFrame([headline_title,headline_link]).T
    headline_news.rename(columns = {0:'Headline News Title',1:'Link'},inplace=True)
    # headline_news['Link'] = headline_news['Link'].apply(lambda x : make_hyper(x))
#     folder_path = os.getcwd()
#     headline_news.to_excel('Today News.xlsx')
#     os.startfile(folder_path)



    # def make_hyper(value):
    #     return '=HYPERLINK("%s")' % (value.format(value))
    def make_clickable(val):
        return '<a href="{}">{}</a>'.format(val,val)
    return headline_news.style.format(make_clickable)


print('#'*50)
print('        ',today,"의 리마인더입니다.")
print('#'*50,'\n\n')
print('1.날씨')
print('*'*50)
print("                        오늘")
print('-'*50)
today_weather()
print('*'*50)
print("<미세먼지>")
print('-'*50)
dust()
print('\n')

print('\n')
print('2.코로나19 현재 상황')
print('*'*50,'\n\n')
corona19()
print('3.인기 국내증시 현재 상황')
print('*'*50)
popular_stock()

print('4. News')
NEWS()

##################################################
         2021-07-19-Monday 의 리마인더입니다.
################################################## 


1.날씨
**************************************************
                        오늘
--------------------------------------------------
현재위치 : 경기도 김포시 김포본동 ℃
현재온도 : 29 ℃
최고온도 : 33 ℃
최저온도 : 25 ℃
체감온도 : 31 ℃
구름많음, 어제보다 2˚ 낮아요
오전강수확률 : 20 %
오후강수확률 : 60 %
**************************************************
<미세먼지>
--------------------------------------------------




2.코로나19 현재 상황
************************************************** 


['환자 현황(7.19. 00시 기준', " '20.1.3. 이후 누계)"]
국내발생 : 1,208
누적환자 :  179,203 명
3.인기 국내증시 현재 상황
**************************************************


,종목명,현재가,변화
0,[삼성전자],"[79,100]",[하락]
1,[에코프로에이치엔],"[279,000]",[상승]
2,[에스디바이오센서],"[60,700]",[하락]
3,[카카오],"[155,000]",[하락]
4,[신일전자],"[2,575]",[상승]
5,[진원생명과학],"[52,900]",[상승]
6,[대한전선],"[2,770]",[상승]
7,[에코프로비엠],"[281,000]",[상승]
8,[피에이치씨],"[2,810]",[상승]
9,[카카오게임즈],"[84,300]",[하락]


4. News


,Headline News Title,Link
0,[속보] 코로나19 신규 확진자 역대 일요일 최다 1252명,https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=469&aid=0000618107
1,"원산폭격 영상 올린 교수 ""'내가 조국이다' 바람 불기 원했다""",https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=025&aid=0003119385
2,"尹 ""삼부토건 골프접대 받은 적 없어…악의적 보도""",https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=018&aid=0004988353
3,"'동물은 물건 아니다'...법무부, 민법 개정안 입법예고",https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=469&aid=0000618109
4,"靑 ""23일 한일정상회담? 文대통령 방일 아직 확정된 것 없어""",https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=421&aid=0005485798
